A simple example where a transition node converts one token to many tokens that are then put in the output place node.

This example illustrates how given a transition function that returns a list of a type matching the place node,
an individual token is created for each item in the list. For example, a token representing a bag of marbles is
turned into many tokens representing individual marbles


In [ ]:
from petritype.core.executable_graph_components import (
    ListPlaceNode, FunctionTransitionNode, ArgumentEdgeToTransition, ReturnedEdgeFromTransition,
    ExecutableGraphOperations
)

In [ ]:
from pydantic import BaseModel


class Marble(BaseModel):
    colour: str
    number: int

class BagOfMarbles(BaseModel):
    contents: list[Marble]


In [ ]:
def empty_out_bag(bag: BagOfMarbles) -> list[Marble]:
    return bag.contents


In [ ]:
bag_1 = BagOfMarbles(contents=[
    Marble(colour="red", number=1),
    Marble(colour="blue", number=2),
    Marble(colour="green", number=3),
])

In [ ]:
executable_graph_nodes_and_edges = [
    ListPlaceNode("Box", BagOfMarbles, tokens=[bag_1]),
    ArgumentEdgeToTransition("Box", "Empty Marbles into Bowl", "bag"),
    FunctionTransitionNode("Empty Marbles into Bowl", empty_out_bag),
    ReturnedEdgeFromTransition("Empty Marbles into Bowl", "Bowl"),
    ListPlaceNode("Bowl", Marble)
]

In [ ]:
executable_graph = ExecutableGraphOperations.construct_graph(executable_graph_nodes_and_edges)

In [ ]:
from petritype.core.rustworkx_graph import RustworkxGraph


executable_pydigraph = RustworkxGraph.from_executable_graph(executable_graph)

In [ ]:
from petritype.plotting.simple_graphviz import SimpleGraphvizVisualization
from IPython.display import display, clear_output

display(SimpleGraphvizVisualization.graph(executable_pydigraph))

In [ ]:
import time
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from petritype.plotting.simple_graphviz import SimpleGraphvizVisualization

async for step, diagram, transitions_fired in SimpleGraphvizVisualization.animate_execution_generator(
    executable_graph=executable_graph,
    executable_pydigraph=executable_pydigraph,
):
    clear_output(wait=True)
    print(f"Step {step}")
    display(diagram)
    print(f"Transitions fired: {transitions_fired}")
    if not transitions_fired:
        print("No more transitions to fire. Execution complete.")
        break
    time.sleep(1.0)
    plt.close()